## 集成学习

### tldr;

- Bagging: 关注方差，RF (随机森林），可并行，Boostrap 抽样。
- Boosting: 关注偏差，AdaBoost, XGBoost, GBDT, 串行。

集成学习 (Ensemble Learning), 将多个个体学习器(individual learner)组合起来，通过不同的结合策略来完成学习任务。这些个体学习器之间可以是同质的 (homogeneous) 也可以是异质的 (heterogeneous)。

其主要思想是通过多个弱分类器的组合，是否可以构建成一个强分类器。对于弱分类器(weak learner) 是指泛化性能略优于随机猜想的学习器，如二分类问题上精度略优于 50% 的分类器。

### 1. 数学支持基础

集成学习的数学基础是 Hoeffding 不等式。下面先复习下一个简单的伯努利事件。

做一下简单的分析：假设二分类问题 $ y \in \{-1, 1\}$ 和真实函数 $f$, 假设基于分类器的错误率为 $\epsilon$, 即对每个分类器 $h_i$ 有:

$$
    P(h_i(\textbf{x}) \neq f(\textbf{x})) = \epsilon
$$

假设集成通过简单的投票法结合 T 个分类器，

$$
    H(\textbf{x}) = sign(\sum_{i=1}^{T}h_i(\textbf{x}))
$$

若有超过半数的 ( 大于 $  T/2 $)基分类器正确，则集成分类器就正确。

而个数小于 **$ \lfloor T/2 \rfloor$** 则都被错误分类了。假设基分类器的错误率相互独立，则由 Hoeffding 不等式可知，集成的错误率为：

$$
    P(H(\textbf{x}) \neq f(\textbf{x})) = \sum_{k = 0}^{\lfloor T/2 \rfloor}(_k^T)(1 - \epsilon)^k \epsilon^{T-k} \\
    \leqslant exp(-\frac{1}{2}T(1 - 2\epsilon)^2)
$$


上式显示出，随着 T 的增大，集成的错误率将呈指数级下降，最终走向于零。

### 2. 集成学习分类

目前集成学习可以分成两大类。

1. 一种是个体学习器之间存在强的依赖关系，必须串行生成的序列化方法。代表有： Boosting, XGBoost 。
2. 一种是个体学习器之间不存在强的依赖关系，可以同时并行化生成。代表有：Bagging 和随机森林 （RF, Random Forest）。



### 3. Boosting

Boosting 是一族可以将弱学习器提升为强学习器的算法。其工作机制为：

1. 为训练集中的每一个样本赋予一个权重，这些权重构成了向量 $\textbf{D}$。 权重向量 $\textbf{D}$ 始终满足同一分布，初始化时每个样本的权重都是相等的，如可以取总体样本数的倒数 1/m。

2. 根据前一个学习器 $h_t$，计算其错误率 $\epsilon$, 根据 $\epsilon$ 来更新权重向量 $D_{t+1}$ 。具体来说前一次正确分类的样本权重将降低，而错误分类的权重将增加。

3. 重复训练，直到基学习器的个数达到 T 或者错误率为 0。

4. 最终从 T 个弱分类中组合成一个强分类器，而这 T 个弱分类器也是有不同的权重的，其每个弱分类器对应的权重为 $\alpha $ alpha。

其中错误率的计算公式为：

$$
   \epsilon = \frac{h_i(\textbf{x}) \neq f(\textbf{x})} {m}
$$

而 alpha 的计算公式如下：

$$
    \alpha = \frac{1}{2} ln(\frac{1 - \epsilon}{\epsilon})
$$

计算出 alpha 后，可以对权重向量进行更新：

如果上一个分类器分类正确，则要降低权重，所以：$\textbf{D}_i^{t+1} = \frac{\textbf{D}_i^t e^{-\alpha}}{sum(\textbf{D})}$

如果上一个分类器分类正确，则要增加权重，所以：$\textbf{D}_i^{t+1} = \frac{\textbf{D}_i^t e^{\alpha}}{sum(\textbf{D})}$





Boosting 算法中最著名的代表是 AdaBoost.

#### 3.1 AdaBoost

AdaBoost 算法有多种推导方式，比较容易理解是的基于『加性模型』(additive model)，即基学习器的线性组合：

$$
H(\textbf{x}) = \sum_{t=1}^{T}\alpha_{t}h_t(\textbf{x})
$$

#### 3.2 AdaBoost 算法
- **输入**：
    
    训练集： $X = \{(x_1, y_1), (x_2, y_2), ...,  (x_m, y_m)\}$ 

    学习算法： $\Omega$ 

    训练迭代数：T

- **算法**

```python
D(X) = 1 / m

for t = 1,2, ..., T do:
    h_t = Ω(X, D)
    ε = (h_t(x) ≠ f(x))
    
    if ε > 0.5 then: break
    
    ⍺ = 1/2 * ln((1-ε)/ε)
    
    根据 ⍺ 更新 D.
end for
```
- **输出：** $H(\textbf{x}) = sign(\sum_{t=1}^{T}\alpha_t h_t(\textbf{x}))$



从理论上来说，$sign(H(\textbf{x}))$ 达到了**贝叶斯最优错误率**。

Boosting 算法要求基学习器能对特定的数据分布进行学习，这样每一轮可以进行重新赋于新的权重 （re-weighting）来进行下一轮学习。

对于无法接受带权重样本的基学习法，则可以通过重新采样 (re-sampling) 来处理，即在每一轮学习中，根据样本分布对训练集重新进行采样，再用重采样而得的样本集对基学习器进行训练。一般而言，这两种做法没有显著的优劣差别。

从偏差-方差分解的角度来看，Boosting 主要关注降低偏差，因此 Boosting 能基于泛化能力相当弱的学习器构建出很强的集成。

自己可以根据**决策树桩**来实现一个集成学习器。

### 4. Bagging 和随机森林

自助采样法，Bootstrap Aggregating ，也称为 Bagging 方法，是从原始数据集选择 S 次后得到 S 个新的数据集。然后根据每一个训练数据集训练出一个基学习器，最后再将这些基学习器进行结合。这就是 Bagging 的基本流程。

对于分类问题，Bagging 组合一般采取简单的投票法；而对于回归问题，一般采用简单的平均法。

#### 4.1 自助采样法 （Bootstrap Sampling）
自助采样法，即有放回采样。 给定有 m 个样本的样本集 D, 对基生成采样生成样本集 D': 每次从 D 中挑选一个样本拷贝到 D' 中，进行有放回采样，即采样的样本空间不会减少。这样进行 m 次后，样本在 m 次没有被采到的概率为 $(1 - \frac{1}{m})^m$, 对其取极限得到：

$$
\lim_{x \rightarrow \infty}(1 - \frac{1}{m})^m \rightarrow  \frac{1}{e} \approx \frac{1}{2.718} \approx 0.368
$$

通过自助采样，初始数据集 D 中约有 36.8% 的样本未出现在 D' 中，我们可以将 D' 作为训练集；而 D - D' 作为测试集。这样，实际评估的模型与期望评估的模型都使用 m 个训练样本，而我们仍有数据量约 1/3 的、没在训练集中出现的样本用于测试，这样的测试结果，也称为『包外估计』(out-of-bag estimate).


#### 4.2 Bagging 算法
- **输入**：
    
    训练集： $X = \{(x_1, y_1), (x_2, y_2), ...,  (x_m, y_m)\}$ 

    学习算法： $\Omega$ 

    训练迭代数：T

- **算法过程**

```python
for t = 1,2, ..., T do:
    h_t = Ω(X, D)
end for
```
- **输出：** $H(\textbf{x}) = arg \max_{y \in Y} \sum_{t=1}^{T} (h_t(\textbf{x}) = y)$

假设基学习器的复杂度为 O(m)， 则 Bagging 的复杂度大致为 T(O(m) + O(s)), 抽样过程 O(s), 和 T 都是一个不大的常数，所以 Bagging 集成和基学习算法的复杂度是同阶的。这说明 Bagging 是一个很高效的集成算法。另外，与**标准的 AdaBoost 只适用于二分类任务**不同，Bagging 能不经修改地用于多分类，回归等任务。

Bagging 还可以应用『包外估计』(out-of-bag estimate). 包外样本还有其它用途，如当基学习器是决策树时，可以使用包外样本来辅助剪枝；当基学习器是神经网络时，可以使用包外样本辅助早期停止以减少过拟合风险。

从偏差-方差分解的角度来看，Bagging 主要关注的是**降低方差**，因此它在不剪枝决策树、神经网络等易受样本扰动的学习器上效果更为明显。


#### 4.3 随机森林（Random Forest, RF）

随机森林，RF 是 Bagging 的一个变体。RF 在以决策树为基学习器构建 Bagging 集成的基础上，进一步在决策树的训练过程中**引入了随机属性选择**。

具体来说，传统决策树在划分时是从所有的属性中(假设有 d 个属性)选择一个最优属性。而在 RF 中，**是从 k (k <= d) 个属性集中选择一个最优属性。若 k = d ，则基决策树和传统决策树没有什么不同，一般 $k = \log_2{d}$**。

RF 实现简单、计算开销小，令人惊奇的是它在很多现实任务中都表现出强大的性能，被誉为：集成学习方法的代表。

可以看出 RF 不仅对样本多样性进行了扰动，而且还对属性进行了扰动。

注意：RF 和 Bagging 的起始性能相对较差；然而随着个体学习器数目的增加，RF 通常会收敛到更低的泛化误差。另外，RF 的效率要优于 Bagging, 而 RF 每次划分的属性集更小。

### 5. 结合策略
假定集成包含 T 个基学习器 $\{h_1, h_2, ..., h_T\}$, 其中 $h_i$ 在样本 $\textbf{x}$ 上的输出为 $h_i(\textbf{x})$。则对 $h_i$ 有下面几种常见的结合策略。

#### 5.1 平均法 （回归）
平均法一般用回归，对于数值型输出 $h_i(\textbf{x}) \in \mathbb{R} $有一般平均法和加权平均法。

一般平均法：

$$
H(\textbf{x}) = \frac{1}{T}\sum_{i=1}^{T} h_i(\textbf{x})
$$

加权平均法：

$$
H(\textbf{x}) = \frac{1}{T}\sum_{i=1}^{T} w_ih_i(\textbf{x})
$$

#### 5.2 投票法 (分类)
#### 5.3 学习法(如： stacking)
我们把个体学习器称为初级学习器，而用于结合的学习器称为次级学习器和元学习器(meta-learner)。


### 6. 多样性
构建泛化能力强的集成，个体学习应 『好而不同』。这里的不同就是多样性。

多样性增强的方法有：

1. 数据样本扰动
2. 输入属性扰动
3. 输出表示扰动
4. 算法参数扰动


#### 7. 总结
Boosting 主要关注降低偏差；而 Bagging 则关注降低方差。

#### 8. 扩展
AdaBoost 的推导基于『统计视角 (statistical view)』, 此派理论认为 AdaBoost 以类似牛顿迭代法来优化指数损失函数。受此启发，将迭代优化过程换成其它优化方法，就产生了 **Gradient Boosting** 方法。